In [1]:
import pandas as pd

# GET

We will explore a dataset from the Vanderbilt Department of Biostatistics that involves the prevalence of obesity, diabetes, and other cardiovascular risk factors. Obtaining the data involves first pulling the *.csv* from the school's [online datastore](https://hbiostat.org/data).

In [2]:
data = pd.read_csv("https://hbiostat.org/data/repo/diabetes.csv")


We'll also save a copy of the original dataset courtesy of the Vanderbilt Department of Biostatistics, in case the webpage is down or relocated.

In [3]:
data.to_csv("diabetes.csv")

Let's have our first glance at the dataset by looking at the first 5 rows.

In [4]:
data.head()

,id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,weight,frame,bp.1s,bp.1d,bp.2s,bp.2d,waist,hip,time.ppn
0,1000,203.0,82,56.0,3.6,4.31,Buckingham,46,female,62.0,121.0,medium,118.0,59.0,NaN,NaN,29.0,38.0,720.0
1,1001,165.0,97,24.0,6.9,4.44,Buckingham,29,female,64.0,218.0,large,112.0,68.0,NaN,NaN,46.0,48.0,360.0
2,1002,228.0,92,37.0,6.2,4.64,Buckingham,58,female,61.0,256.0,large,190.0,92.0,185.0,92.0,49.0,57.0,180.0
3,1003,78.0,93,12.0,6.5,4.63,Buckingham,67,male,67.0,119.0,large,110.0,50.0,NaN,NaN,33.0,38.0,480.0
4,1005,249.0,90,28.0,8.9,7.72,Buckingham,64,male,68.0,183.0,medium,138.0,80.0,NaN,NaN,44.0,41.0,300.0


First, we are able to remove the `id` variable, as it provides no additional information for our exploration and model crafting. Pandas provides a sufficient means of indexing should we require it.

In [5]:
data.drop("id", axis="columns", inplace=True)

Next, let's take a look at the `location` variable. From the dataset description, we know most of these individuals are from Virginia.

In [6]:
for location in data.location.unique():
    print(location)

Buckingham
Louisa


It seems like there are only two counties in the dataset -- Buckingham, and Louisa. Especially considering these are counties of the same state, it is unlikely the patients exact county location will have that large of an impact on the final model, so we will remove the variable altogether.

In [7]:
data.drop("location", axis="columns", inplace=True)

Lastly, let's take a look at how we might handle null variables throughout the dataset.

In [8]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   chol      402 non-null    float64
 1   stab.glu  403 non-null    int64  
 2   hdl       402 non-null    float64
 3   ratio     402 non-null    float64
 4   glyhb     390 non-null    float64
 5   age       403 non-null    int64  
 6   gender    403 non-null    object 
 7   height    398 non-null    float64
 8   weight    402 non-null    float64
 9   frame     391 non-null    object 
 10  bp.1s     398 non-null    float64
 11  bp.1d     398 non-null    float64
 12  bp.2s     141 non-null    float64
 13  bp.2d     141 non-null    float64
 14  waist     401 non-null    float64
 15  hip       401 non-null    float64
 16  time.ppn  400 non-null    float64
dtypes: float64(13), int64(2), object(2)
memory usage: 53.6+ KB
None


So, it seems there is quite a few missing values throughout the dataset. Most variables have only a handful or so missing, but blood pressure measurements are the most notable, with around half of the second blood pressure measurements never taken. It's likely that the first in-office blood pressure reading is [misleadingly high](https://www.heart.org/en/news/2018/05/01/dont-just-get-your-bp-taken-make-sure-its-taken-the-right-way). Travel to involves physical exertion, stress, pain, or extreme heat or cold, so a patient may arrive with initially high blood pressure, but may have a lower blood pressure normally. For this reason, best practice involves discarding all but the last taken measurement.

In [9]:
data["bp.s"] = data["bp.2s"].fillna(data["bp.1s"])
data["bp.d"] = data["bp.2d"].fillna(data["bp.1d"])

By filling in the missing values for the second blood pressure reading with the first blood pressure reading, we ensure only the latest systolic and diastolic reading are kept -- in line with the best practices.

In [10]:
data[["bp.s", "bp.d", "bp.1s", "bp.1d", "bp.2s", "bp.2d"]].head(10)

,bp.s,bp.d,bp.1s,bp.1d,bp.2s,bp.2d
0,118.0,59.0,118.0,59.0,NaN,NaN
1,112.0,68.0,112.0,68.0,NaN,NaN
2,185.0,92.0,190.0,92.0,185.0,92.0
3,110.0,50.0,110.0,50.0,NaN,NaN
4,138.0,80.0,138.0,80.0,NaN,NaN
5,132.0,86.0,132.0,86.0,NaN,NaN
6,161.0,112.0,161.0,112.0,161.0,112.0
7,NaN,NaN,NaN,NaN,NaN,NaN
8,128.0,86.0,160.0,80.0,128.0,86.0
9,108.0,72.0,108.0,72.0,NaN,NaN


Accordingly, we will drop the duplicated or misleading blood pressure information by removing the two individual readings, and just keeping the last.

In [11]:
data.drop(["bp.1s", "bp.1d", "bp.2s", "bp.2d"], axis="columns", inplace=True)

However, such as the case with the 7th index, there are still missing values across blood pressure readings and other variables.

In [12]:
print(data.count())

chol        402
stab.glu    403
hdl         402
ratio       402
glyhb       390
age         403
gender      403
height      398
weight      402
frame       391
waist       401
hip         401
time.ppn    400
bp.s        398
bp.d        398
dtype: int64


In [13]:
data["glyhb"].isna().sum()

13

It seems there are a fair number of missing glycosolated gemoglobin readings. Considering this is our target variable, it seems wise to completely drop any rows without a `glyhb` value.

In [14]:
data = data[data["glyhb"].notna()]

Now we are left with missing values throughout each of the variables. Let's look at these rows to get a better understanding of what's missing.

In [15]:
data[data.isna().any(axis="columns")]

,chol,stab.glu,hdl,ratio,glyhb,age,gender,height,weight,frame,waist,hip,time.ppn,bp.s,bp.d
7,227.0,75,44.0,5.2,3.94,37,male,59.0,170.0,medium,34.0,39.0,1020.0,NaN,NaN
13,183.0,79,46.0,4.0,4.59,40,female,59.0,165.0,medium,37.0,43.0,60.0,NaN,NaN
27,NaN,74,NaN,NaN,4.28,48,male,68.0,100.0,small,27.0,33.0,510.0,120.0,85.0
37,190.0,107,32.0,5.9,3.56,46,male,72.0,205.0,medium,46.0,49.0,240.0,NaN,NaN
50,196.0,82,58.0,3.4,4.25,76,male,65.0,154.0,NaN,37.0,41.0,120.0,140.0,84.0
63,145.0,85,29.0,5.0,3.99,38,female,NaN,125.0,NaN,31.0,35.0,120.0,NaN,NaN
69,232.0,184,114.0,2.0,8.40,91,female,61.0,127.0,NaN,35.0,38.0,120.0,170.0,82.0
86,218.0,182,54.0,4.0,10.55,51,female,NaN,215.0,large,42.0,53.0,720.0,139.0,69.0
108,292.0,235,55.0,5.3,7.87,79,male,70.0,165.0,NaN,39.0,41.0,240.0,170.0,100.0
110,244.0,101,36.0,6.8,4.66,32,male,70.0,212.0,NaN,39.0,44.0,NaN,132.0,90.0


There are no immediately obvious patterns to the missing data. These missing values will not interfere too much during our exploratory data analysis, but may impact our modeling. It is likely that only a subset of these variables will be used for our final model, so we will keep the remaining missing values as they are. Then, during model exploration, we will address missing values when we have finalized our model. 

We save off the cleaned diabetes dataset, ready for us to begin our exploratory data analysis.

In [16]:
data.to_csv("cleaned-diabetes.csv", index=False)